# WiDS Datathon 2021 - Clean Feature Selection

Target : Area under the Receiver Operating Characteristic (ROC) curve between the predicted and the observed target (diabetes_mellitus_diagnosis).

by : Sebastián Uribe Ocampo

## Libraries

In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns
sns.set_style("whitegrid")
#import plotly.express as px
# pd.options.plotting.backend = "plotly"
pd.set_option('display.max_rows', 60)
pd.set_option('display.max_columns', 100)

In [3]:
from scipy.stats import chi2_contingency

# UTILS

In [4]:
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

# LOAD DATA

In [74]:
data_info = pd.read_csv("../docs/DataDictionaryWiDS2021.csv",sep=";")
demographic_features = data_info[data_info["Category"]=="demographic"].sort_values(["Data Type","Variable Name"])["Variable Name"].unique().tolist()
demographic_features.remove("icu_admit_type")
apache_comorbidity_features = data_info[data_info["Category"]=="APACHE comorbidity"].sort_values(["Data Type","Variable Name"])["Variable Name"].unique().tolist()
apache_covariate_features =  data_info[data_info["Category"]=="APACHE covariate"].sort_values(["Data Type","Variable Name"])["Variable Name"].unique().tolist()
vitals_features =  data_info[data_info["Category"]=="vitals"].sort_values(["Data Type","Variable Name"])["Variable Name"].unique().tolist()
labs_features =  data_info[data_info["Category"]=="labs"].sort_values(["Data Type","Variable Name"])["Variable Name"].unique().tolist()
labs_bloodgas_features =  data_info[data_info["Category"]=="labs blood gas"].sort_values(["Data Type","Variable Name"])["Variable Name"].unique().tolist()

# Clean

In [75]:
data_info["Data Type"].value_counts()

numeric    151
binary      15
string       9
integer      6
Name: Data Type, dtype: int64

In [94]:
data_info[data_info["Data Type"]=="binary"]["Variable Name"].unique()

array(['elective_surgery', 'readmission_status', 'apache_post_operative',
       'arf_apache', 'gcs_unable_apache', 'intubated_apache',
       'ventilated_apache', 'aids', 'cirrhosis', 'hepatic_failure',
       'immunosuppression', 'leukemia', 'lymphoma',
       'solid_tumor_with_metastasis', 'diabetes_mellitus'], dtype=object)

In [118]:
categorical_data = ['ethnicity', 'gender', 'hospital_admit_source','icu_admit_source', 'icu_stay_type', 'icu_type','apache_2_diagnosis', 'apache_3j_diagnosis']
dtype_dict = {}
for var in data_info["Variable Name"].unique():
    if data_info[data_info["Variable Name"]==var]["Data Type"].iloc[0]=="binary" :
        pass#dtype_dict[var]=pd.Int64Dtype()
    if data_info[data_info["Variable Name"]==var]["Data Type"].iloc[0]=="string" :
        dtype_dict[var]="object"
    if data_info[data_info["Variable Name"]==var]["Data Type"].iloc[0]=="numeric" :
        dtype_dict[var]="float64"
    if data_info[data_info["Variable Name"]==var]["Data Type"].iloc[0]=="integer" :
        pass#dtype_dict[var]=pd.Int64Dtype()
    if var in categorical_data :
        dtype_dict[var]="category"
        
dtype_dict.pop('hospital_id', None)

In [119]:
dtype_dict

{'age': 'float64',
 'bmi': 'float64',
 'ethnicity': 'category',
 'gender': 'category',
 'height': 'float64',
 'hospital_admit_source': 'category',
 'icu_admit_source': 'category',
 'icu_admit_type': 'object',
 'icu_stay_type': 'category',
 'icu_type': 'category',
 'pre_icu_los_days': 'float64',
 'weight': 'float64',
 'albumin_apache': 'float64',
 'apache_2_diagnosis': 'category',
 'apache_3j_diagnosis': 'category',
 'bilirubin_apache': 'float64',
 'bun_apache': 'float64',
 'creatinine_apache': 'float64',
 'fio2_apache': 'float64',
 'glucose_apache': 'float64',
 'heart_rate_apache': 'float64',
 'hematocrit_apache': 'float64',
 'map_apache': 'float64',
 'paco2_apache': 'float64',
 'paco2_for_ph_apache': 'float64',
 'pao2_apache': 'float64',
 'ph_apache': 'float64',
 'resprate_apache': 'float64',
 'sodium_apache': 'float64',
 'temp_apache': 'float64',
 'urineoutput_apache': 'float64',
 'wbc_apache': 'float64',
 'd1_diasbp_invasive_max': 'float64',
 'd1_diasbp_invasive_min': 'float64',
 'd

# DTypes

In [120]:
diabetes = pd.read_csv("../data/raw/TrainingWiDS2021.csv",dtype=dtype_dict)

In [122]:
diabetes.dtypes

Unnamed: 0                       int64
encounter_id                     int64
hospital_id                      int64
age                            float64
bmi                            float64
                                ...   
immunosuppression                int64
leukemia                         int64
lymphoma                         int64
solid_tumor_with_metastasis      int64
diabetes_mellitus                int64
Length: 181, dtype: object

In [123]:
print("Train File Size   : ",diabetes.shape)
print("# encounter_id    : ",diabetes.encounter_id.nunique())
print("# hospital_id     : ",diabetes.hospital_id.nunique())

Train File Size   :  (130157, 181)
# encounter_id    :  130157
# hospital_id     :  204


In [124]:
diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130157 entries, 0 to 130156
Columns: 181 entries, Unnamed: 0 to diabetes_mellitus
dtypes: category(8), float64(155), int64(18)
memory usage: 172.9 MB


## Drop Columns

In [125]:
diabetes.head()

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,...,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
0,1,214826,118,68.0,22.732803,0,Caucasian,M,180.3,Floor,Floor,92,admit,CTICU,0.541667,0,73.9,2.3,113,502.01,0,0,0.4,31.0,2.51,NaN,3.0,6.0,0.0,4.0,168.0,118.0,27.4,0,40.0,NaN,NaN,NaN,NaN,36.0,134.0,39.3,NaN,0,14.1,46.0,32.0,68.0,37.0,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,1
1,2,246060,81,77.0,27.421875,0,Caucasian,F,160.0,Floor,Floor,90,admit,Med-Surg ICU,0.927778,0,70.2,NaN,108,203.01,0,0,NaN,9.0,0.56,1.0,1.0,3.0,0.0,1.0,145.0,120.0,36.9,0,46.0,37.0,37.0,51.0,7.45,33.0,145.0,35.1,NaN,1,12.7,NaN,NaN,95.0,31.0,95.0,...,9.0,9.0,8.6,8.6,0.56,0.56,145.0,143.0,27.0,27.0,11.3,11.3,36.9,36.9,1.3,1.3,3.5,3.5,557.0,557.0,4.2,4.2,145.0,145.0,12.7,12.7,37.0,37.0,7.45,7.45,51.0,51.0,54.8,51.000000,37.0,37.0,7.45,7.45,51.0,51.0,51.0,51.0,0,0,0,0,0,0,0,1
2,3,276985,118,25.0,31.952749,0,Caucasian,F,172.7,Emergency Department,Accident & Emergency,93,admit,Med-Surg ICU,0.000694,0,95.3,NaN,122,703.03,0,0,NaN,NaN,NaN,NaN,3.0,6.0,0.0,5.0,NaN,102.0,NaN,0,68.0,NaN,NaN,NaN,NaN,37.0,NaN,36.7,NaN,0,NaN,NaN,NaN,88.0,48.0,88.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
3,4,262220,118,81.0,22.635548,1,Caucasian,F,165.1,Operating Room,Operating Room / Recovery,92,admit,CTICU,0.000694,0,61.7,NaN,203,1206.03,1,0,NaN,NaN,NaN,0.6,4.0,6.0,0.0,5.0,185.0,114.0,25.9,1,60.0,30.0,30.0,142.0,7.39,4.0,NaN,34.8,NaN,1,8.0,62.0,30.0,48.0,42.0,48.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.6,11.6,34.0,34.0,1.6,1.1,NaN,NaN,43.0,43.0,NaN,NaN,NaN,NaN,8.8,8.8,37.0,27.0,7.44,7.34,337.0,102.0,342.5,236.666667,36.0,33.0,7.37,7.34,337.0,265.0,337.0,337.0,0,0,0,0,0,0,0,0
4,5,201746,33,19.0,NaN,0,Caucasian,M,188.0,NaN,Accident & Emergency,91,admit,Med-Surg ICU,0.073611,0,NaN,NaN,119,601.01,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,NaN,0,103.0,NaN,NaN,NaN,NaN,16.0,NaN,36.7,NaN,0,NaN,NaN,NaN,99.0,57.0,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0


In [126]:
diabetes=diabetes.drop(columns=["Unnamed: 0","hospital_id"])

In [127]:
diabetes.isnull().mean().sort_values(ascending=False).head()

h1_bilirubin_max    0.920896
h1_bilirubin_min    0.920896
h1_albumin_min      0.914319
h1_albumin_max      0.914319
h1_lactate_max      0.910185
dtype: float64

In [128]:
invasive_features = list(filter(lambda x:"_invasive_" in x,diabetes.columns))

In [129]:
diabetes=diabetes.drop(columns=invasive_features)

In [130]:
diabetes[demographic_features].isnull().sum().sort_values(ascending=False)

hospital_admit_source    33198
age                       4988
bmi                       4490
weight                    3463
height                    2077
ethnicity                 1587
icu_admit_source           240
gender                      66
icu_type                     0
icu_stay_type                0
pre_icu_los_days             0
icu_id                       0
readmission_status           0
elective_surgery             0
dtype: int64

In [131]:
diabetes[apache_comorbidity_features].isnull().sum().sort_values(ascending=False)

solid_tumor_with_metastasis    0
lymphoma                       0
leukemia                       0
immunosuppression              0
hepatic_failure                0
cirrhosis                      0
aids                           0
dtype: int64

In [132]:
diabetes[apache_covariate_features].isnull().sum().sort_values(ascending=False)

paco2_for_ph_apache      99720
ph_apache                99720
fio2_apache              99720
paco2_apache             99720
pao2_apache              99720
bilirubin_apache         82560
albumin_apache           78163
urineoutput_apache       63167
wbc_apache               29475
hematocrit_apache        26758
bun_apache               25411
creatinine_apache        24882
sodium_apache            24519
glucose_apache           14696
temp_apache               6611
gcs_eyes_apache           2190
gcs_motor_apache          2190
gcs_verbal_apache         2190
apache_2_diagnosis        1685
apache_3j_diagnosis        865
resprate_apache            808
gcs_unable_apache          709
map_apache                 420
heart_rate_apache          309
ventilated_apache            0
intubated_apache             0
arf_apache                   0
apache_post_operative        0
dtype: int64

In [133]:
type(pd.Series([1, None], dtype="Int64")[1])

pandas._libs.missing.NAType

In [135]:
diabetes[apache_covariate_features].corr().replace(1.0,0).style.highlight_max(color='lightgreen').highlight_min(color='red')

,apache_post_operative,arf_apache,gcs_unable_apache,intubated_apache,ventilated_apache,gcs_eyes_apache,gcs_motor_apache,gcs_verbal_apache,albumin_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,glucose_apache,heart_rate_apache,hematocrit_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,wbc_apache
apache_post_operative,0.000000,-0.026631,0.044790,0.183352,0.168669,0.007249,0.017923,-0.011943,-0.062950,-0.012607,-0.164515,-0.103490,-0.003487,-0.033376,-0.063530,-0.127547,-0.002054,-0.063506,-0.063506,0.083565,0.040568,-0.164687,0.055566,-0.045615,0.037657,0.088408
arf_apache,-0.026631,0.000000,-0.004647,-0.002146,0.002961,-0.002756,-0.001909,-0.000823,-0.023261,0.001263,0.181941,0.521205,0.001326,0.005395,-0.011534,-0.102081,0.001820,-0.012221,-0.012221,0.016579,0.020171,0.003500,-0.058361,-0.021346,-0.109485,-0.011854
gcs_unable_apache,0.044790,-0.004647,0.000000,0.120449,0.135531,nan,nan,nan,-0.014851,0.012059,0.000373,-0.001417,0.047687,0.011394,0.010764,-0.010689,-0.004786,0.003874,0.003874,0.027891,-0.025057,-0.025522,0.020766,-0.043540,-0.008176,0.031225
intubated_apache,0.183352,-0.002146,0.120449,0.000000,0.613449,-0.386175,-0.356360,-0.436794,-0.114507,0.022498,0.024820,0.024481,0.310543,0.053680,0.081341,-0.080862,-0.018677,-0.115694,-0.115694,0.253725,-0.019547,-0.016908,0.080732,-0.130109,-0.008481,0.138118
ventilated_apache,0.168669,0.002961,0.135531,0.613449,0.000000,-0.467585,-0.422268,-0.537863,-0.138098,0.010210,0.055981,0.033534,0.283371,0.063942,0.119402,-0.082915,-0.023304,0.083955,0.083955,0.178147,-0.062622,0.023362,0.098385,-0.137113,-0.014226,0.158157
gcs_eyes_apache,0.007249,-0.002756,nan,-0.386175,-0.467585,0.000000,0.799371,0.777415,0.090287,-0.031667,-0.048257,-0.041328,-0.142296,-0.059384,-0.088563,0.020477,-0.007871,0.045586,0.045586,-0.092605,0.072709,-0.002940,-0.105912,0.153446,0.035179,-0.110511
gcs_motor_apache,0.017923,-0.001909,nan,-0.356360,-0.422268,0.799371,0.000000,0.703082,0.086599,-0.025043,-0.049171,-0.044638,-0.159496,-0.070541,-0.082531,0.018423,-0.008038,0.053606,0.053606,-0.089872,0.099710,-0.008998,-0.101250,0.193573,0.031547,-0.111993
gcs_verbal_apache,-0.011943,-0.000823,nan,-0.436794,-0.537863,0.777415,0.703082,0.000000,0.113289,-0.028746,-0.066154,-0.036635,-0.148302,-0.042970,-0.109818,0.045585,-0.009060,0.063062,0.063062,-0.104549,0.033404,-0.015747,-0.129954,0.125112,0.052443,-0.120937
albumin_apache,-0.062950,-0.023261,-0.014851,-0.114507,-0.138098,0.090287,0.086599,0.113289,0.000000,-0.134835,-0.207615,-0.113871,-0.110090,-0.028858,-0.167257,0.419918,0.148418,0.109495,0.109495,0.023285,0.139615,-0.040062,-0.011621,0.056733,0.137936,-0.158166
bilirubin_apache,-0.012607,0.001263,0.012059,0.022498,0.010210,-0.031667,-0.025043,-0.028746,-0.134835,0.000000,0.115324,0.077907,0.034079,-0.046612,0.052214,-0.138607,-0.060952,-0.113811,-0.113811,-0.003025,-0.032307,0.028761,-0.106201,-0.029867,-0.079040,0.049780


## Rows

## OutLiers

## Clean PipeLine